In [ ]:
import os
import wget
from RAG.system import RAGSystem
from hyperparameters import best
from dotenv import load_dotenv

In [ ]:
load_dotenv()
DATA_DIR = os.getenv("DATA_DIR")
assert DATA_DIR is not None, "Define DATA_DIR in .env file!"

In [ ]:
rag = RAGSystem(
    hypers=best,
    data_dir=DATA_DIR,
    random_seed=17,
)

In [ ]:
rag.ask('What is the capital of France?')

In [ ]:
rag.ask('What is the capital of France?', force_answer=True, debug=True)